In [15]:
import librosa
import numpy as np
from tensorflow.keras.models import load_model
from pydub import AudioSegment
import os
from python_speech_features import mfcc
import shutil

#input audio file
input= "input.wav"
#output folder to save clips and mfccs
output_folder = "output"

# creating a folder is output folder is not existing
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

#loading the audio file
audio = AudioSegment.from_wav(input)

#get the length of the audio clip
duration = len(audio)

#setting clip length
clip_length = 5000

#splitting to 5 seconds clips
for i in range(0, duration, clip_length):
    start_time = i
    end_time = i + clip_length
    
    clip = audio[start_time:end_time]
    
    #save the clips in the folder
    output_file = os.path.join(output_folder, f"clip{i//clip_length}.wav")
    clip.export(output_file, format="wav")

for file_name in os.listdir(output_folder):
    if file_name.endswith('.wav'):
        #loading the audio file
        audio_path = os.path.join(output_folder, file_name)
        audio, sr = librosa.load(audio_path, sr=44100)

        #processing the audio clips to 5 seconds
        duration = 5 # in seconds
        samples = sr * duration
        audio = librosa.resample(audio, orig_sr=sr, target_sr=22050)
        if len(audio) > samples:
            audio = audio[:samples]
        else:
            audio = np.pad(audio, (0, samples - len(audio)), mode='constant')

        #extract the features
        mfccs = librosa.feature.mfcc(y=audio, sr=22050, n_mfcc=13, n_fft=2048, hop_length=512)

        #fave the mfccs
        mfcc_path = os.path.join(output_folder, file_name.replace('.wav', '.npy'))
        np.save(mfcc_path, mfccs)

mfccs = []
for file_name in os.listdir(output_folder):
    if file_name.endswith('.npy'):
        #load the mfccs
        mfcc_path = os.path.join(output_folder, file_name)
        mfcc = np.load(mfcc_path)
        # appending the mfccs to the list
        mfccs.append(mfcc)

#convert the list to a numpy array
mfccs = np.array(mfccs)

#loading the trained model
model = load_model('model3.h5')

#Making the predictions
predictions = model.predict(mfccs)

def average_prediction(predictions):
    total = 0
    for prediction in predictions:
        total += prediction
    return total / len(predictions)

final_prediction = (average_prediction(predictions))*100

os.remove(input)
shutil.rmtree(output_folder)
# Open a file in write mode
file = open("results.txt", "w")
prediction_print = "{:.2f}".format(final_prediction[0])

# Write some text to the file
file.write(f"Depression percentage: {prediction_print}\n")
if(final_prediction>85):
    file.write("You are depressed")
elif(final_prediction>50):
    file.write("You are more likely to be depressed")
else:
    file.write("You are less likely to be depressed")

# Close the file
file.close()

1/1 [==============================] - 1s 921ms/step


In [9]:
final_prediction = (average_prediction(predictions))*100
print(final_prediction[0])

0.0003456325


In [10]:
os.remove(input)
shutil.rmtree(output_folder)

In [11]:
# Open a file in write mode
file = open("results.txt", "w")
prediction_print = "{:.2f}".format(final_prediction[0])

# Write some text to the file
file.write(f"Depression percentage: {prediction_print}\n")
if(final_prediction>85):
    file.write("You are depressed")
elif(final_prediction>50):
    file.write("You are more likely to be depressed")
else:
    file.write("You are less likely to be depressed")

# Close the file
file.close()